# Control Systems 1, NB03: Time Response and Stability
2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It introduces how to determine the output response and stability of an LTI system.

Authors:
- Jonas Holinger; jholinger@ethz.ch
- Shubham Gupta; shugupta@ethz.ch


## Learning Objectives

After completing this notebook, you should be able to:

1. Understand how to compute the general form response of a linear system by combining its initial condition response and its forced response.
2. Understand the various definitions of stability for a dynamical system.
3. Understand whether an LTI system is asymptotically stable, Lyapunov stable, BIBO stable or unstable by looking at the eigenvalues of the A Matrix

# Setup

## Installing the required packages:


In [ ]:
%pip install numpy matplotlib scipy ipywidgets control IPython

## Import the packages
The following cell imports the required packages. Run it before running the rest of the notebook.

In [2]:
import control as ct
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from scipy.integrate import odeint
from ipywidgets import interactive
from IPython.display import display, clear_output, Math

# 1. Output Response of a Linear System


Thus far we have seen how to determine the governing equations of a system, how to represent them into standard (state-space) form, and determine whether the system is Linear, Time-Invariant, Causal and Dynamic/Static. Thus, a natural follow-up is how do we compute the output of such a system. 

As previously covered, in this course we consider mainly LTI systems, and thus the following approach details how to compute the output of an LTI system. 

From previous lectures we learned that the definition of linear system is: 
$$
\Sigma (\alpha u_a + \beta u_b) = \alpha (\Sigma u_a) + \beta (\Sigma u_b) = \alpha y_a + \beta y_b
$$
Thus, the idea is to express the output signal as a linear combination of simpler input signals.

Due to causality, we know that the effects of the previous inputs $u_{past}(t)$ can be summarized by the state $x(t)$ at time $t$.
Furthermore, due to time-invariance, the time reference does not matter, hence we can pick any $t$. It follows that it is simpler to pick $t=0$, as this enables us to study separately the effects of non-zero inputs and of non-zero initial conditions. Hence, we are interested in the following two-types of responses:

- **Initial-conditions response:**

  $$
  \left\{
  \begin{array}{ll}
  x_{IC}(0) = x_0 \\
  u_{IC}(t) = 0
  \end{array}
  \right.
  \quad \longrightarrow \quad y_{IC};
  $$

- **Forced response:**

  $$
  \left\{
  \begin{array}{ll}
  x_{F}(0) = 0 \\
  u_{F}(t) = u(t)
  \end{array}
  \right.
  \quad \longrightarrow \quad y_{F}
  $$


The general output of a linear system can be expressed as: $$y = y_{IC}+y_{F}$$

In general note that the solutions to the above can be generalized to:

- **Initial Condition:**
    The initial-condition problem is
    $
    \begin{align}
    \dot x(t) &= A x(t), \quad x(0) = 0 \\
    y(t) &= C x(t)
    \end{align}
    $
    With the corresponding general solution:
    $
    \begin{align}
    x(t) &= e^{At} x(0) \\
    y_{IC}(t) &= C e^{At} x(0)
    \end{align}
    $

- **Forced Response:**
    The forced response problem is:
    $$
    \begin{align}
    \dot x(t) &= Ax(t) + Bu(t), \quad x(0) = 0 \\
    y(t) &= Cx(t) + Du(t)
    \end{align}
    $$
    With the corresponding general solution:
    $$
    \begin{align}
    x(t) &= \int_{0}^t e^{A(t-\tau)Bu(\tau)d\tau} \\
    y_{F}(t) &= \int_{0}^t Ce^{A(t-\tau)Bu(\tau)d\tau} + Du(t)
    \end{align}
    $$

Below, we provide a couple examples first, computing the above for a scalar linear system. However, since the $A$ matrix is, in general, not scalar, we later on illustrate how to compute the forced response for when $A$ is a matrix. 

## 1.1 Example: Output Response of a Scalar Linear System


As an example of a scalar linear system lets consider a room with temperature $T(t)$, thermal capacity $C$, and with a heater with power $P(t)$. The room loses heat as a rate of $G(T(t)-T_{amb})$, where $G$ is the thermal conductance. The system is described by the following: 
$$C \frac{dT(t)}{dt} + G \left(T(t) - T_{\text{amb}}\right) = P(t)$$

<div style="text-align:center;">
    <img src="./media/sys4iofo.png" alt="Block Diagram" width="500">
</div>


We choose $x(t) = T(t)$ as our state and $u(t) = P(t)$ as our control input. Further, as winter is approaching, let $T_{amb} = 0$. Thus, the state-space representation of the system is:
$$
\begin{align}
a &= -\frac{G}{C}, &b&= \frac{1}{C} \\
c &= 1, &d&= 0
\end{align}
$$
*Note that we typically use capatilized letters (e.g., $A,B,C,D$) for matrices, and lowercase letters (e.g., $a,b,c,d$) for scalars.* 



In [51]:
#initalise the system

# Define a time vector
time = np.linspace(0, 20, 500)

#Define the param
C = 2
G = 3

# Define state-space system
def create_system(G, C):
    A = -G / C
    B = 1 / C
    C_matrix = 1
    D = 0
    return ct.StateSpace(A, B, C_matrix, D)

# func for generating input
def generate_input(input_type, time_vector):
    if input_type == 'Constant':
        return np.full_like(time_vector, forced_input_slider.value)
    elif input_type == 'Ramp (5s)':
        ramp_duration = 5
        ramp_input = np.clip(np.linspace(0, forced_input_slider.value, int(ramp_duration / time[1])),
                              0, None)
        constant_input = np.full_like(time_vector[int(ramp_duration / time[1]):], forced_input_slider.value)
        return np.concatenate((ramp_input, constant_input))
    elif input_type == 'Ramp (15s)':
        ramp_duration = 15
        ramp_input = np.clip(np.linspace(0, forced_input_slider.value, int(ramp_duration / time[1])),
                              0, None)
        constant_input = np.full_like(time_vector[int(ramp_duration / time[1]):], forced_input_slider.value)
        return np.concatenate((ramp_input, constant_input))
    elif input_type == 'Sinusoidal':
        return forced_input_slider.value * np.sin(0.5 * np.pi * time_vector)  # Frequency can be adjusted

### 1.1.1 Initial Condition Response


To get the initial condition response for the system we need to solve the following ODE: (Let $T_0$ denote the temperature at T=0)
$$
\dot{x}(t)_{IC} = ax(t)_{IC}, 
$$
$$
y(t)_{IC}=cx(t)_{IC}
$$
$$
x(0) = x_0 = T_0
$$

The general solution is known from Analysis :
$$
x(t)_{IC} = e^{at}T_0
$$

$$
y(t)_{IC} = ce^{at}T_0
$$
For our specific problem we get:
$$
y(t)_{IC} = e^{\frac{-G}{C}}T_0
$$


#### Visualization


In the Visualization you can see how the state of our system exponentially decays to zero.
Try changing the Values of $T_0, G, C$, does the system react as you would expect?

In [ ]:
# Initialize a global Math object for LaTeX rendering
latex_display = display(Math(r"y(t) = e^{-\frac{1.0}{1.0} t} \cdot 1.0"), display_id=True)

# Define the update_plot_ic function
def update_plot_ic(T_0, G, C):
    clear_output(wait=True)  # Clear previous output
    
    # Update system and response
    system = create_system(G, C)
    time = np.linspace(0, 10, 100)  # Example time array
    t, y = ct.initial_response(system, T=time, X0=[T_0])

    # Update the LaTeX formula
    G_str = f"{G:.2f}"
    C_str = f"{C:.2f}"
    t_0_str = f"{T_0:.2f}"
    formula = fr"y(t) = {t_0_str} \cdot e^{{-\frac{{{G_str}}}{{{C_str}}} t}}"
    
    # Update the LaTeX display without re-creating it
    latex_display.update(Math(formula))

    # Plot the system response
    plt.figure()
    plt.plot(t, y, label='Initial Response', color='blue')
    plt.title('System Response to Initial Condition')
    plt.xlabel('Time')
    plt.ylabel('Temperature')
    plt.grid()
    plt.legend()
    plt.show()

# Example sliders for G, C, and t_0
G_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='G:')
C_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='C:')
T_0_slider = widgets.FloatSlider(value=1.0, min=-20, max=20, step=0.5, description='T₀:')

# Interactive plot
interactive_plot = interactive(
    update_plot_ic, 
    T_0=T_0_slider,
    G=G_slider,
    C=C_slider,
)

# Display sliders and initial plot
display(interactive_plot)

### 1.1.2 Forced Response


The forced response, which was derived in the lecture with $u(t) \neq 0$ and $x_0 = 0$ is:
$$
\begin{align}
x_{F}&=\int_{0}^{t}e^{a\cdot (t-\tau )}bu(\tau )d\tau \\
y_{FC}&=\int_{0}^{t}ce^{a\cdot (t-\tau )}bu(\tau )d\tau + du(t)
\end{align}
$$

#### Visualization
In the visualization you can see how the system reacts to different input types, which can be selected by the drop-down menu. The input slider adjusts how strong the amplitude of the input is. You can show the response due to initial condition and the total response by pressing the buttons.

In [ ]:
# Initialize a display object
output = widgets.Output()

# Function to update the plot
def update_plot_fc(T_0, G, C, input_type, show_initial_response, show_total_response):
    with output:
        clear_output(wait=True)  
        system = create_system(G, C)  
        time = np.linspace(0, 10, 100)  # Example time array
        
        # Simulate the response to the current initial condition
        t, initial_response = ct.initial_response(system, T=time, X0=[T_0])
        
        # Generate the forced input
        input_fc = generate_input(input_type, time)
        
        # Use control's forced_response to get output for the input signal
        t, forced_response = ct.forced_response(system, T=time, U=input_fc)
        
        # Total response
        total_response = initial_response + forced_response
        
        # Plot the forced input response
        plt.figure(figsize=(10, 5))
        plt.plot(t, forced_response, label='Forced Output', color='red')
        plt.plot(t, input_fc, label='Forced Input', color='orange', linestyle='--')
        
        # Plot initial response only if the checkbox is on
        if show_initial_response:
            plt.plot(t, initial_response, label='Initial Response', color='green')
            
        # Plot total response only if the checkbox is on
        if show_total_response:
            plt.plot(t, total_response, label='Total Response', color='blue')

        plt.title('System Response')
        plt.xlabel('Time (s)')
        plt.ylabel('Temperature')
        plt.ylim(min(min(forced_response), min(input_fc)), max(max(forced_response), max(input_fc)))
        plt.grid()
        plt.legend()
        plt.show()

# Create sliders for G, C, T₀
G_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='G:')
C_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='C:')
T_0_slider = widgets.FloatSlider(value=1.0, min=-15.0, max=40.0, step=0.1, description='T₀:')

# Create input type dropdown and checkboxes for responses
input_type_dropdown = widgets.Dropdown(
    options=['Constant', 'Ramp (5s)', 'Ramp (15s)', 'Sinusoidal'],
    value='Ramp (5s)',
    description='Input Type:'
)

show_initial_response_button = widgets.Checkbox(
    value=False,
    description='Show Initial Response',
)

show_total_response_button = widgets.Checkbox(
    value=False,
    description='Show Total Response',
)

# Arrange widgets in two columns using HBox and VBox
left_column = widgets.VBox([T_0_slider, G_slider, C_slider])
right_column = widgets.VBox([input_type_dropdown, show_initial_response_button, show_total_response_button])

# Use HBox to put both columns side by side
ui = widgets.HBox([left_column, right_column])

# Link the widgets to the update_plot function
widgets.interactive_output(update_plot_fc, {
    'T_0': T_0_slider,
    'G': G_slider,
    'C': C_slider,
    'input_type': input_type_dropdown,
    'show_initial_response': show_initial_response_button,
    'show_total_response': show_total_response_button
})

# Display the controls and the plot
display(ui, output)

# 2 Output Response of General Linear System


Recall that for an LTI System, the output $y(t)$ is the sum of the initial and forced responses:
$$
\begin{align}
y_{IC}(t) &= Ce^{At}x_0 \\
y_{F}(t) &= \int_{0}^t Ce^{A(t-\tau)Bu(\tau)d\tau} + Du(t) \\
y(t) &= y_{IC}(t) + y_{F}(t)
\end{align}
$$
Previously we computed the above for scalar cases. However, when $A,B,C,D$ are matrices, how can one compute the above? Most notably, how do we compute the matrix exponential $e^{At}$? Below we detail how to compute such terms. 

## 2.1 Diagonalization and Jordan Forms


1. One approach is to use the following Taylor Expansion.
$$
e^{At} = I + At + \frac{1}{2} (At)^2 + \frac{1}{3!} (At)^3 + \dots + \frac{1}{n} (At)^n
$$
However, this is time-consuming, and it may be difficult to estimate how many terms to compute. 

2. In python, use the function `numpy.expm(A)`.
3. Find a realization of the system such that the matrix $A$ is either:
   1. A **diagonal matrix**, with the diagonal elements being the eigenvalues of $A$ -- i.e., $A = \begin{bmatrix} \lambda_1 & 0 \\0 & \lambda_2 \end{bmatrix}$, where $\lambda_{1,2}$ are the eigenvalues of $A$. The matrix exponential is then:
   $$
   e^{At} = \begin{bmatrix} e^{\lambda_1 t} & 0 \\ 0 & e^{\lambda_2 t} \end{bmatrix}
   $$
   2. A **Jordan form matrix**, where $A$ has repeated eigenvalues -- i.e., $A = \begin{bmatrix} \lambda & 1 \\ 0 & \lambda \end{bmatrix}$, where $\lambda$ is the repeated eigenvalue of $A$. The matrix exponential is then:
   $$
   e^{At} = \begin{bmatrix} e^{\lambda t} & t e^{\lambda t} \\ 0 & e^{\lambda t} \end{bmatrix}
   $$
This last step enables a simpler approach towards computing the matrix exponential. However, it not necessarily the case that $A$ is a diagonal matrix of eigenvalues (or a Jordan form) -- think back to the system modelling we performed in NB01 and PS01. Notwithstanding, it is in general possible to transform a diagonalizable $A$ matrix of an LTI system such that a corresponding transformed matrix $\tilde{A}$ is a diagonal matrix of eigenvalues. In the following section we detail this procedure.

## 2.2 Similarity Transformations


Consider that provided with an LTI system:
$$
\begin{align}
\dot x(t) &= Ax(t) + Bu(t)\\
y(t) &= Cx(t) + Du(t)
\end{align}
$$
In the case that $A$ is not a diagonal or Jordan form matrix, we wish transform $A$ such that $\tilde{A}$ is a diagonal matrix. 

To do so, we can use the fact that the choice of a state space model is not unique for a system, and hence we can transform it to a more suitable form for calculating the matrix exponential by hand. 

One method is to use similarity transforms. Let $T$ denote an invertible matrix such that it performs the following transforms: $x = T \tilde{x}, \tilde{x} = T^{-1}x$. Then the standard state-space model can be rewritten as:

$$
\begin{cases}
T\dot{\tilde{x}} = AT\tilde{x} + Bu, \\
y = CT\tilde{x} + Du.
\end{cases}
\quad
$$

Re-arrange to standard form:

$$
\begin{cases}
\dot{\tilde{x}} &= (T^{-1} A T) \tilde{x} + (T^{-1} B) u &= \tilde{A} \tilde{x} + \tilde{B} u, \\
y &= (CT) \tilde{x} + Du &= \tilde{C} \tilde{x} + \tilde{D} u.
\end{cases}
$$

Note that the time response of this transformed system is the same for both $A,B,C,D$ and $(\tilde{A},\tilde{B},\tilde{C},\tilde{D})$.

The above transformation is useful because if $A$ is diagonalizable (i.e., it has $n$ linearly independent eigenvectors), then it is possible to select a $T$ such that: $A = T\Delta T^{-1}$, where $\Delta = \tilde{A}$ denotes the diagonalized matrix of eigenvalues.
*Note that $T$ is in general a matrix whose columns represent the eigenvectors of $A$*. 

Now that we have a mechanism by which to transform a diagonalizable matrix $A$ into one of the standard forms, we can now compute the output response by either:
1. Computing the output response of the transformed system, using $\tilde{A}, \tilde{B}, \tilde{C}, \tilde{D}$. 
2. Computing the matrix exponential using $\tilde{A}$, and then transforming this back to the original coordinates -- i.e., $e^{At} = T e^{\tilde{A}t} T^{-1}$. 

To help contextualize this procedure, below we provide an example using the first option. 

### 2.2.1 Example
Suppose we wish to compute the initial condition output response of an LTI state space model with initial condition $x(0)$ given by the following matrices.
$$\begin{array}{c} A = \begin{bmatrix} 0 & 1 \\ -2 & -3 \end{bmatrix}, \quad B = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, \quad C = \begin{bmatrix} 1 & 0 \end{bmatrix}, \quad D = 0, \quad x(0) =  \begin{bmatrix} 2 \\ 3 \end{bmatrix} \end{array}$$

It is immediately noticeable that matrix $A$ is not in either of the 2 preferred forms. Thus, we must diagonalize it. 

**Step 1:** Find the eigenvalues of A

Solve: $\quad\det(\lambda I - A ) = 0$

to get $\lambda_{1}  = -1$ and $\lambda_{2} = -2$.

**Step 2:** Find the eigenvectors of A

Solve $(\lambda I - A)v = 0$ 

To get the eigenvectors $ \begin{bmatrix}
1 \\ -1
\end{bmatrix}$ and $\begin{bmatrix}
-1 \\ 2
\end{bmatrix}$
which gives us the matrix $ T = \begin{bmatrix}
1 &  -1\\
-1 &  2\\
\end{bmatrix}
$. 

**Step 3:** Apply Similarity Transform to the state-space model

$$
\begin{align}
\tilde{A} &= T^{-1} A T &=& \begin{bmatrix} -1 & 0 \\ 0 & -2 \end{bmatrix} \quad 
&\tilde{B}& = T^{-1} B = \begin{bmatrix} 1 \\ 1 \end{bmatrix} \\
\tilde{C} &= C T &=& \begin{bmatrix} 1 & -1 \end{bmatrix} \quad
&\tilde{D}& = 0
\end{align}
$$

$$\tilde{x}_0 = T^{-1}x_0 = \begin{bmatrix} 7 \\ 5 \end{bmatrix}$$
It is visible that transformed matrix $\tilde{A}$ is a diagonal matrix with the eigenvalues of matrix $A$.

**Step 4:** Initial Condition Output Response

The state response due to the initial condition is given by: $\tilde{x}(t) = e^{\tilde{A}t}\tilde{x}_{0}$ 
$$\begin{bmatrix}
\tilde{x}_1 \\ \tilde{x}_2 
\end{bmatrix} = \begin{bmatrix}
e^{-t} &  0\\ 0
&  e^{-2t}\\ 
\end{bmatrix}\begin{bmatrix} 7 \\ 5 \end{bmatrix}$$

The initial condition output response is given by: 
$$
\begin{align*}
y_{IC}(t) = \tilde{C} e^{At} = \begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} 7e^{-t} \\ 5e^{-2t} \end{bmatrix} = 7e^{-t}
\end{align*}
$$


## 2.3 Modal Decomposition

Since the eigenvectors $v_i, i = 1,\dots, n$ form a basis, we can express **any** initial condition $x(0)$ as a linear combination of eigenvectors: $x(0) = V \tilde{x}(0)$, where $V$ denotes the matrix of eigenvectors -- notationally, this is equal to $T$ in the above --, and hence: 
$$
x(t) = \sum_{i=1}^n e^{\lambda_i t}\tilde{x}_i(0)v_i
$$

**Self-assessment**: Try performing the exercise above but using this form instead, you should note that the returned time responses are the same. 

## 2.4 Quick Recap: The Euler Formula



Note that in the case of complex eigenvalues, the below formula may come in use:

$$e^{i\phi} = \cos \phi + i \sin \phi$$

#### Visualisation
For a more intuitiv understanding of the euler formula, you can see how sin and cos change with respect to $\phi$

In [ ]:
def update_plot_img(phi):
    plt.figure(figsize=(8, 8))
    
    # Create unit circle
    circle = plt.Circle((0, 0), 1, color='blue', fill=False, linestyle='dotted')
    
    fig, ax = plt.subplots()
    ax.add_artist(circle)

    x = np.cos(phi)
    y = np.sin(phi)

    # Plotting the point on the unit circle
    ax.plot([0, x], [0, y], color='red', linewidth=2, label='Radius: e^{iφ}')
    ax.plot(x, y, 'ro')  # Point on the circle

    # Plot the cosine and sine functions
    t = np.linspace(0, 2 * np.pi, 100)
    ax.plot([0, math.cos(phi)], [0, 0], color='green', linestyle='--', linewidth=2, label='cos')
    ax.plot([0, 0], [0, math.sin(phi)], color='blue', linestyle='--', linewidth=2, label='sin')
    # Formatting the plot
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.axhline(0, color='black', linewidth=0.5, ls='--')
    ax.axvline(0, color='black', linewidth=0.5, ls='--')
    ax.set_aspect('equal', 'box')
    ax.set_title(f'$e^{{i{phi:.2f}}} = \\cos({phi:.2f}) + i\\sin({phi:.2f})$', fontsize=14)
    ax.legend()
    plt.grid()
    plt.show()

phi_slider = widgets.FloatSlider(value=0, min=0, max=2 * np.pi, step=0.1, description='φ (radians)')
widgets.interactive(update_plot_img, phi=phi_slider)


# 3. Stability


The notion of stability is central in the study of control theory. Previously this has been introduced intuitively. Think back to the examples seen in Lecture 1 (SN2401), Lecture 2 (NB01) and Lecture 3 (NB02) -- where the expected/desired output was/could not be achieved. Below, the notion of stability is formalized a bit further, providing qualitative mechanisms to analyze an LTI system: 

To a reasonable extent, in this course we refer to stability in the sense of Lyapunov -- i.e., if you start close to an equilibrium point, you can never go far from it:

1. **Lyapunov Stability**: A system is called Lyapunov stable if, for any initial condition $x_0$ bounded by $\epsilon$, and zero-input $u(t)=0, \forall t$, then the state $x(t)$ of the system remains bounded by $\delta$, such that if $||x(0)-x_e||<\delta$ then for every $t\geq 0$, $||x(0)-x_e||<\epsilon$.
$$ 
    ||x(0)||\leq\epsilon\;,\; u(t) = 0 \implies ||x(t)||\leq\delta,\quad\forall t\geq 0
$$

A stronger condition to Lyapunov Stability, is stating the asymptotic behavior of the system as time progresses:

2. **Asymptotically stable**: A system is called asymptotically stable if for any initial condition $x_0$ bounded by $\epsilon$, and zero-input $u(t)=0, \forall t$, the system state $x(t)$ converges to origin (equilibrium point) as time progresses:
$$
    ||x(0)||\leq\epsilon\;,\;u(t) = 0 \; \implies\; \lim_{t\rightarrow\infty}||x(t)|| = 0
$$
If the above holds for all possible initial conditions $x_0$, then the system is *globally asymptotically stable* -- otherwise its *locally asymptotically stable*.

Up until now we have defined types of system stability without input. However, by nature of having a system we wish to control, it can be useful to know whether for a given (bounded) input, the output remains bounded:  
1. **BIBO stability**: A system is called *Bounded Input Bounded Output stable*, if for any bounded input the system will produce an bounded output
$$ 
    ||u(t)||\leq\epsilon,\quad\forall\;t\geq 0\;,\: x_0=0\;\implies||y(t)||\leq\delta,\quad\forall\;t\geq 0
$$

In general, we will rely on the above to determine stability of LTI systems, however, when designing real-life systems, it is important to keep in mind that the linear system may be an approximation of the non-linear system. In such cases, it is important to note the following **Hartman-Grobman Theorem**:
> The qualitative behavior of a non-linear system about some hyperbolic equilibrium point $(x_e, u_e)$ is given by the qualitative behavior of a linear system about the same equilibrium point.

In terms of stability, this implies that if the linearization of a nonlinear system around an equilibrium point is asymptotically stable (or unstable), then this equilibrium is a locally asymptotically stable (or unstable) equilibrium of the nonlinear system as well.

**NOTE**: The Hartman-Grobman Theorem only applies to hyperbolic equilibrium points. An equilibrium point is hyperbolic if all eigenvalues of the linearization have non-zero real parts (i.e., $\textrm{Re}(\lambda_i)\neq 0, \forall i$). Thus, no claims about Lyapunov stability (i.e. where $\mathrm{Re}(\lambda_i)\leq 0$) can be made. 

The rest of this section is dedicated towards detailing how to determine whether a system is stable, as well as a worked-through example.

### 3.1 Conditions For stability


For a finite-dimensional LTI system:
$$
    \dot{x} = Ax + Bu \\
    y = Cx + Du
$$

Stability is determined by the matrix $A$. The conditions for stability are as follows, $\lambda_i$ denoting the eigenvalues of $A$:
1. If $A$ is diagonalizable $\implies$ Lyapunov stable if $Re(\lambda_i)\leq 0$, Asymptotically stable if $Re(\lambda_i)<0, \forall i$.
2. If $A$ is not diagonalizable $\implies$ Lyapunov stable if $Re(\lambda_i)\leq 0$ for all $i$, and no repeated eigenvalues with 0 real part
3. Otherwise, unstable.

TODO: Add exmples for diff stability types in the from of graphs

### 3.2 Example: Pendulum Stability



The equations for a pendulum can be derived as follows using Newtonian Mechanics:
$$
    \ddot{\theta} +\frac{g}{l}sin(\theta) = 0
$$

This equation can be converted to the standard state-space form considering the following state vector:
$$
    x = \begin{bmatrix} x_1 \\ x_2\end{bmatrix} = \begin{bmatrix} \theta\\\dot{\theta}\end{bmatrix}
$$
Using this the state space equations are:
$$
    \dot{x} = \begin{bmatrix} \dot{x_1} \\ \dot{x_2}\end{bmatrix} = \begin{bmatrix} \dot{\theta|}\\\ddot{\theta}\end{bmatrix} = \begin{bmatrix} x_2 \\ -\frac{g}{l}sin(x_1)\end{bmatrix}
$$

The zero-input equilibrium points for the system can be found using $\dot{x} = 0$, which comes out to be 
$$
    sin(x_1) = sin(\theta) = 0 \\
    \implies \theta = n\pi\; \forall \; n = \{0, \pm 1,\pm 2,\pm 3, \pm 4,\ldots \}
$$

On a closer look, we can see that these are essentially just two equilibrium points, being repeated with a $2\pi$ rotation. The two equillibria being at the bottom ($x_1 = 0$) and the vertical top ($x_1 = \pi$)

Linearizing the system around these two equilibrium points we get:

For $x_1 = 0$ equilibrium point (bob at the bottom):
$$
    \dot{x} = \begin{bmatrix} \dot{x_1} \\ \dot{x_2}\end{bmatrix} = \begin{bmatrix} \dot{\theta|}\\\ddot{\theta}\end{bmatrix} = \begin{bmatrix} 0 & 1 \\ -\frac{g}{l} & 0\end{bmatrix} x
$$

Using this formulation the eigenvalues of the $A$ matrix come out to be $\lambda_{1,2}=\pm i\sqrt(10)$. Since we have two purely imaginary eigenvalues (i.e., $Re(\lambda_i)=0, \forall i$), the system is Lyapunov stable at this equilibrium point, but not asymptotically stable. Intuitively, this makes sense since without air drag, any $x_0\neq0$ will result in symmetrical oscillations. 

For $x_1 = \pi$ equilibrium point (bob at the top):
$$
    \dot{x} = \begin{bmatrix} \dot{x_1} \\ \dot{x_2}\end{bmatrix} = \begin{bmatrix} \dot{\theta|}\\\ddot{\theta}\end{bmatrix} = \begin{bmatrix} 0 & 1 \\ \frac{g}{l} & 0\end{bmatrix} x
$$

Using this formulation the eigenvalues of the $A$ matrix come out to be $\lambda_{1,2} = \pm\sqrt(10)$. Since one of the eigenvalues has real part greater than 0, the system is unstable at this equilibrium. Intuitively, this makes sense, as for any $x_0 \neq pi$, the pendulum will.

# 4. Spring and Pendulum System


In this section we provide an example that qualitatively illustrates the notion of stability, and the Hartman-Grobman Theorem. To do so, we consider the same pendulum example as in NB02.

### Refresher


Consider a pendulum that is mounted to a wall and connected to a horizontal spring as below.
Let $l$ be the length of the pendulum, $J$ its moment of inertia, $m$ its mass, and let $\lambda$ represent the damping constant of the pendulum acting at the pivot (damped rotation due to friction).

Further, let $k$ represent the spring constant of the spring, and denote by $a$ the distance to the pivot of the point that connects the spring and the pendulum. Assume the spring to be relaxed at $\varphi =0$.
The system is actuated by an external force $F(t)$ which acts at a right angle to the pendulum.
There is a sensor measuring the angle $\varphi$ which we assume to be limited to $\varphi(t)\in\left(-\frac{\pi}{2},\frac{\pi}{2}\right)$.

<div style="text-align:center;">
    <img src="./media/pendel.png" alt="Block Diagram" width="400">
</div>

We linearize the system about the equilibrium point $\varphi = 0$, $\dot \varphi = 0$ and $F(t) = 0$.

The linearized system is given by:

$$
\begin{array}{rcl}
\dot{x}(t) &=& \begin{bmatrix} 0 & 1 \\ -\frac{mgl + a^2 k}{J} & -\frac{\lambda}{J} \end{bmatrix} x(t) + \begin{bmatrix} 0 \\ \frac{l}{J} \end{bmatrix} u(t) \\[0.5em]
y(t) &=& \begin{bmatrix} 1 & 0 \end{bmatrix} x(t)
\end{array}
$$
with the following parameters:
Further, let the system parameters be
$$
\begin{array}{rcl}
\begin{array}{rcl}
l &=& 1\text{ m}  & m &=& 1\text{ kg} \\[0.2em]
g &=& 10\text{ m/s}^2  & a &=& 0.5\text{ m} \\[0.2em]
k &=& 10\text{ N/m} & \lambda &=& 3\text{ Nms/rad} \\[0.2em]
J &=& 1\text{ Nms}^2/\text{rad} & &
\end{array}
\end{array}
$$


## Interactive Example

The time response from both the linearized and non-linear systems are plotted below. 
- What type of stability is present in the below? Lyapunov Stability, local asymptotic stability, or unstable.
- What do you notice about the system behavior of both the linear and non-linear systems? 
- Try varying both $\varphi, d\varphi/ dt$ sliders. What do you notice when the slider values are distant from their initial (equilibrium) points?

In this example we would like to place the pendulum in a specific location. In Lecture 1, it was taught that adding feedback may help improve performance, and thus we add a simple feedback loop with gain $K$.
- Try varying $K$, can you find a range of value of $K$ for which the linear (and non-linear) systems are:
  - Locally asymptotically stable. 
  - Lyapunov stable. 
  - Unstable.  

In [ ]:
# System parameters
l = 1.0  # m
m = 1.0  # kg
g = 10.0  # m/s^2
a = 0.5  # m
k = 10.0  # N/m
lambda_ = 3.0  # Nms/rad
J = 1.0  # Nms^2/rad

# Control input F(t)
def F(t, K):
    return K 

# Nonlinear system dynamics
def nonlinear_system(x, t, K):
    phi, phi_dot = x
    phi_ddot = (-l * m * g * np.sin(phi) - 0.5 * a**2 * k * np.sin(2*phi) - lambda_ * phi_dot + l * (K* phi + K* phi_dot)) / J
    return [phi_dot, phi_ddot]

# Linear system dynamics
def linear_system(x, t, K):
    phi, phi_dot = x
    phi_ddot = (-l * m * g * phi - a**2 * k * phi - lambda_ * phi_dot + l * (K* phi + K* phi_dot)) / J
    return [phi_dot, phi_ddot]

# Simulation and plotting function
def simulate_and_plot(phi0, phi_dot0, K):
    # Time array
    t = np.linspace(0, 30, 1000)

    # Initial conditions
    x0 = [phi0, phi_dot0]

    # Solve ODEs
    nonlinear_solution = odeint(nonlinear_system, x0, t, args=(K,))
    linear_solution = odeint(linear_system, x0, t, args=(K,))

    # Extract phi and phi_dot
    nonlinear_phi = nonlinear_solution[:, 0]
    linear_phi = linear_solution[:, 0]

    # Plotting
    fig, (ax1) = plt.subplots(1, 1)

    # Plot phi
    ax1.plot(t, nonlinear_phi, label='Nonlinear')
    ax1.plot(t, linear_phi, label='Linear', linestyle='--')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Angle (rad)')
    ax1.set_title('Pendulum System Simulation: Nonlinear vs Linear')
    ax1.grid(True)
    ax1.legend()

    plt.tight_layout()
    plt.subplots_adjust(top=0.85, wspace=0.3)
    plt.show()

# Create widgets
K_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='K:')
phi0_slider = widgets.FloatSlider(value=0.0, min=-np.pi/2, max=np.pi/2, step=0.01, description='ϕ₀:')
phi_dot0_slider = widgets.FloatSlider(value=0.0, min=-5, max=5, step=0.01, description='dϕ₀/dt:')

# Create interactive output
interactive_plot = widgets.interactive(simulate_and_plot,
                                       phi0=phi0_slider, phi_dot0=phi_dot0_slider, 
                                       K =K_slider)

# Display the interactive plot
display(interactive_plot)